# Ordenamiento de datos

In [7]:
import sys
import os
from functools import reduce

project_path = os.path.abspath('..')
sys.path.insert(1, project_path)

import pandas as pd
import numpy as np

from src.utils import get_project_root 

Variables presentes en ambas estaciones
- SO2
- NO2
- NO
- NOX
- O3
- CO
- Dirección del viento
- Velocidad del viento

In [8]:
def adjust_index(minma_df):
    """Combina los indices fecha y hora de un df con datos de MINMA
    y los transforma en un DatetimeIndex.
    
    Parameters
    ----------
    minma_df : pd.DataFrame
        pd.DataFrame con datos de MINMA a modificar.
    """    
    padding = '0'
    date_len = 6
    time_len = 4 
    minma_df.index = minma_df.index.set_levels([s.rjust(date_len,padding) for s in minma_df.index.levels[0].astype(str)], level=0)
    minma_df.index = minma_df.index.set_levels([s.rjust(time_len,padding) for s in minma_df.index.levels[1].astype(str)], level=1)
    minma_df.index = minma_df.index.map(' '.join)
    minma_df.index = minma_df.index.map(lambda x: '20'+x if (x[0] != '9') else '19'+x) 
    minma_df.index = pd.to_datetime(minma_df.index)

In [9]:
def get_minma_data(param_list, station, from_last=None, to_date=-1, n_cols=5):
    """Recibe una lista de parámetros para una estación y entrega
    un DataFrame con datos dentro de algún período.

    Parameters
    ----------
    param_list : list[str]
        list con parámetros monitoreados a utilizar.
    station : str
        str con nombre de la estación de interés.
    from_last : str
        str indicando el tiempo hacia atrás a considerar.
    to_date : str
        str indicando la fecha hacia el presente a considerar.

    Returns
    -------
    pd.DataFrame
        pd.DataFrame con datos de cada parámetro para alguna estación.       
    """        
    param_df_list = []
    for param in param_list:
        path = get_project_root() / 'data' / 'raw' / station / f'{station}_{param}.csv'
        param_df = pd.read_csv(path,
                                sep=';',
                                usecols=range(len(param_list)),
                                index_col=[0,1]
                                ).add_suffix(f'_{param}')
        param_df_list.append(param_df)
    station_df = reduce(lambda  left,right: pd.merge(left,right,on=['FECHA (YYMMDD)', 'HORA (HHMM)']), param_df_list)
    adjust_index(station_df)
    if to_date == -1:
        to_date = station_df.index[-1]
    from_date = None
    if from_last != None:
        from_date = to_date - pd.Timedelta(from_last)
    return station_df.loc[from_date:to_date]

## Los Maitenes

In [4]:
get_minma_data(['SO2','NO2','NO','NOX','O3','CO'], 'maitenes')

c:\users\nicol\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Registros validados_SO2,Registros preliminares_SO2,Registros no validados_SO2,Registros validados_NO2,Registros preliminares_NO2,Registros no validados_NO2,Registros validados_NO,Registros preliminares_NO,Registros no validados_NO,Registros validados_NOX,Registros preliminares_NOX,Registros no validados_NOX,Registros validados_O3,Registros preliminares_O3,Registros no validados_O3,Registros validados_CO,Registros preliminares_CO,Registros no validados_CO
2009-09-01 01:00:00,NaN,"1,9084",NaN,NaN,"8,2",NaN,NaN,3,NaN,NaN,"11,2",NaN,NaN,"2,6",NaN,NaN,"0,49",NaN
2009-09-01 02:00:00,NaN,"1,14504",NaN,NaN,"8,09998",NaN,NaN,"3,1",NaN,NaN,"11,2",NaN,NaN,"2,4",NaN,NaN,"0,5",NaN
2009-09-01 03:00:00,NaN,"0,381679",NaN,NaN,"7,6",NaN,NaN,"3,2",NaN,NaN,"10,9",NaN,NaN,"2,4",NaN,NaN,"0,5",NaN
2009-09-01 04:00:00,NaN,"0,381679",NaN,NaN,"7,3",NaN,NaN,"3,2",NaN,NaN,"10,5",NaN,NaN,"2,5",NaN,NaN,"0,48",NaN
2009-09-01 05:00:00,NaN,"0,381679",NaN,NaN,"7,3",NaN,NaN,"2,9",NaN,NaN,"10,2",NaN,NaN,"3,1",NaN,NaN,"0,46",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-06 19:00:00,NaN,NaN,"1,72",NaN,NaN,"4,45",NaN,NaN,"2,63838",NaN,NaN,"7,08718",NaN,NaN,29,NaN,NaN,"0,25"
2022-09-06 20:00:00,NaN,NaN,"1,73",NaN,NaN,4,NaN,NaN,"2,3956",NaN,NaN,"6,3959",NaN,NaN,25,NaN,NaN,"0,24"
2022-09-06 21:00:00,NaN,NaN,"1,76",NaN,NaN,"3,71",NaN,NaN,"2,62483",NaN,NaN,"6,33514",NaN,NaN,28,NaN,NaN,"0,23"
2022-09-06 22:00:00,NaN,NaN,"1,73",NaN,NaN,"4,26",NaN,NaN,"2,49475",NaN,NaN,"6,75858",NaN,NaN,28,NaN,NaN,"0,2"


## Quintero

In [5]:
get_minma_data(['SO2','NO2','NO','NOX','O3','CO'], 'quintero')

,Registros validados_SO2,Registros preliminares_SO2,Registros no validados_SO2,Registros validados_NO2,Registros preliminares_NO2,Registros no validados_NO2,Registros validados_NO,Registros preliminares_NO,Registros no validados_NO,Registros validados_NOX,Registros preliminares_NOX,Registros no validados_NOX,Registros validados_O3,Registros preliminares_O3,Registros no validados_O3,Registros validados_CO,Registros preliminares_CO,Registros no validados_CO
2016-02-01 01:00:00,NaN,"1,75957",NaN,NaN,NaN,"2,71378",NaN,NaN,"1,0277",NaN,NaN,"3,68271",NaN,NaN,"5,62378",NaN,NaN,"0,0976166"
2016-02-01 02:00:00,NaN,"1,81287",NaN,NaN,NaN,"3,79096",NaN,NaN,"1,19868",NaN,NaN,"4,91127",NaN,NaN,"3,45831",NaN,NaN,"0,130333"
2016-02-01 03:00:00,NaN,"2,4793",NaN,NaN,NaN,"3,85914",NaN,NaN,"1,1349",NaN,NaN,"4,9156",NaN,NaN,"2,85624",NaN,NaN,"0,0951166"
2016-02-01 04:00:00,NaN,"4,21441",NaN,NaN,NaN,"4,35712",NaN,NaN,"1,2977",NaN,NaN,"5,56602",NaN,NaN,"2,31188",NaN,NaN,"0,0856999"
2016-02-01 05:00:00,NaN,"3,43705",NaN,NaN,NaN,"4,25243",NaN,NaN,"1,65403",NaN,NaN,"5,81372",NaN,NaN,"2,91755",NaN,NaN,"0,0700998"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-06 19:00:00,NaN,NaN,"2,21",NaN,NaN,"6,98",NaN,NaN,"1,35353",NaN,NaN,"8,33331",NaN,NaN,36,NaN,NaN,"0,35"
2022-09-06 20:00:00,NaN,NaN,"2,25",NaN,NaN,"5,5",NaN,NaN,"1,30148",NaN,NaN,"6,80131",NaN,NaN,39,NaN,NaN,"0,31"
2022-09-06 21:00:00,NaN,NaN,"2,27",NaN,NaN,"4,48",NaN,NaN,"1,19537",NaN,NaN,"5,67241",NaN,NaN,39,NaN,NaN,"0,32"
2022-09-06 22:00:00,NaN,NaN,"2,3",NaN,NaN,"3,92",NaN,NaN,"1,09673",NaN,NaN,"5,01439",NaN,NaN,39,NaN,NaN,"0,27"
